In [0]:
import tensorflow as tf
tf.enable_eager_execution()
import numpy as np

from tensorflow.keras.datasets.fashion_mnist import load_data

In [0]:
# noise's shape = [None, 784]

def generator(noise, labels):
  with tf.variable_scope("Generator") and tf.device('/device:GPU:0'):
    inputs = tf.keras.layers.concatenate([noise, labels], 1)
    
    first = tf.keras.layers.Dense(7*7*128)(inputs)
    first = tf.keras.layers.BatchNormalization()(first)
    first = tf.keras.layers.LeakyReLU()(first)
    
    second = tf.keras.layers.Reshape((7, 7, 128))(first)
    second = tf.keras.layers.BatchNormalization()(second)
    second = tf.keras.layers.LeakyReLU()(second)
    
    third_origin = tf.keras.layers.Conv2DTranspose(64, kernel_size = (5, 5), strides=(2, 2), padding='same')(second)
    third = tf.keras.layers.BatchNormalization()(third_origin)
    third = tf.keras.layers.LeakyReLU()(third)
    
    forth_1 = tf.keras.layers.Conv2DTranspose(32, kernel_size = (5, 5), strides=(2, 2), padding='same')(third)
    forth_1 = tf.keras.layers.BatchNormalization()(forth_1)
    forth_1 = tf.keras.layers.LeakyReLU()(forth_1)
    
    forth_2 = tf.keras.layers.Conv2DTranspose(32, kernel_size = (5, 5), strides=(2, 2), padding='same')(third_origin)
    forth_2 = tf.keras.layers.BatchNormalization()(forth_2)
    forth_2 = tf.keras.layers.LeakyReLU()(forth_2)
    
    forth = tf.keras.layers.add([forth_1, forth_2])
    outputs = tf.keras.layers.Conv2DTranspose(1, kernel_size=(5, 5), strides=(1, 1), padding='same')(forth)
  return tf.keras.Model(inputs=inputs, outputs=outputs) 

In [0]:
# # noise's shape = [None, 784]

# def generator():
#   with tf.variable_scope("Generator") and tf.device('/device:GPU:0'):
#     inputs = tf.keras.layers.Input((784, ))
#     first = tf.keras.layers.Dense(7*7*128)(inputs)
#     first = tf.keras.layers.BatchNormalization()(first)
#     first = tf.keras.layers.LeakyReLU()(first)
    
#     second = tf.keras.layers.Reshape((7, 7, 128))(first)
#     second = tf.keras.layers.BatchNormalization()(second)
#     second = tf.keras.layers.LeakyReLU()(second)
    
#     third_origin = tf.keras.layers.Conv2DTranspose(64, kernel_size = (5, 5), strides=(2, 2), padding='same')(second)
#     third = tf.keras.layers.BatchNormalization()(third_origin)
#     third = tf.keras.layers.LeakyReLU()(third)
    
#     forth_1 = tf.keras.layers.Conv2DTranspose(32, kernel_size = (5, 5), strides=(2, 2), padding='same')(third)
#     forth_1 = tf.keras.layers.BatchNormalization()(forth_1)
#     forth_1 = tf.keras.layers.LeakyReLU()(forth_1)
    
#     forth_2 = tf.keras.layers.Conv2DTranspose(32, kernel_size = (5, 5), strides=(2, 2), padding='same')(third_origin)
#     forth_2 = tf.keras.layers.BatchNormalization()(forth_2)
#     forth_2 = tf.keras.layers.LeakyReLU()(forth_2)
    
#     forth = tf.keras.layers.add([forth_1, forth_2])
#     outputs = tf.keras.layers.Conv2DTranspose(1, kernel_size=(5, 5), strides=(1, 1), padding='same')(forth)
#   return tf.keras.Model(inputs=inputs, outputs=outputs) 

In [0]:
# input's shape = [None, 28, 28, 1]

def discriminator(images, labels):
  with tf.variable_scope("Discriminator") and tf.device('/device:GPU:0'):  
    images = tf.keras.layers.Input((28, 28, 1))
    labels = tf.keras.layers.Input((10, ))
    
    inputs = tf.keras.layers.concatenate([images, labels], 1)
    first_1 = tf.keras.layers.Conv2D(32, (5, 5), strides=(1, 1), padding='same', activation=tf.nn.leaky_relu)(inputs)
    first_2 = tf.keras.layers.Conv2D(32, (5, 5), strides=(1, 1), padding='same')(inputs)
    first = tf.keras.layers.add([first_1, first_2])
    first = tf.keras.layers.LeakyReLU()(first)
    first = tf.keras.layers.Dropout(0.4)(first)
    
    second = tf.keras.layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', activation=tf.nn.leaky_relu)(first)
    second = tf.keras.layers.Dropout(0.4)(second)
    
    third = tf.keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same', activation=tf.nn.leaky_relu)(first)

    outputs = tf.keras.layers.Flatten()(third)
  return tf.keras.Model(inputs=inputs, outputs=outputs) 

In [0]:
# # input's shape = [None, 28, 28, 1]

# def discriminator():
#   with tf.variable_scope("Discriminator") and tf.device('/device:GPU:0'):  
#     inputs = tf.keras.Input((28, 28, 1))
    
#     first_1 = tf.keras.layers.Conv2D(32, (5, 5), strides=(1, 1), padding='same', activation=tf.nn.leaky_relu)(inputs)
#     first_2 = tf.keras.layers.Conv2D(32, (5, 5), strides=(1, 1), padding='same')(inputs)
#     first = tf.keras.layers.add([first_1, first_2])
#     first = tf.keras.layers.LeakyReLU()(first)
#     first = tf.keras.layers.Dropout(0.4)(first)
    
#     second = tf.keras.layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', activation=tf.nn.leaky_relu)(first)
#     second = tf.keras.layers.Dropout(0.4)(second)
    
#     third = tf.keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same', activation=tf.nn.leaky_relu)(first)

#     outputs = tf.keras.layers.Flatten()(third)
#   return tf.keras.Model(inputs=inputs, outputs=outputs) 

In [0]:
def gen_loss(gen_out):
  return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logit(labels=tf.ones_like(gen_out), logits=gen_out))

def dis_loss(gen_out, dis_out):
  real_loss = tf.nn.sigmoid_cross_entropy_with_logit(labels=tf.ones_like(dis_out), logits=dis_out)
  gen_loss  = tf.nn.sigmoid_cross_entropy_with_logit(labels=tf.zeros_like(gen_out), logits=gen_out)
  
  total_loss = real_loss + gen_loss
  return total_loss

In [0]:
def get_noise(batch_size, n_noise):
  return tf.random.normal([batch_size, n_noise], dtype=tf.float32)

In [0]:
n_classes = 10
batch_size = 100
total_epoch = 100

In [0]:
noise = tf.keras.Input((784, ))
outputs = tf.keras.Input((10, ))
images = tf.keras.Input((28, 28, 1))

In [29]:
gen = generator(noise, outputs)
dis_real = discriminator(images, outputs)
dis_gene = discriminator(gen, outputs)

AttributeError: ignored